In [1]:
import pandas as pd
import numpy as np
#import math
mu,sigma = 1,1
w = []
l_rate =0
lamda=0
vectors = []

In [2]:
def sigmoid(x):
    global w
    X = np.dot(w,x)
    return round(1/(1+np.exp(-1*X)),2)

def mean_normalise(data):
    global mu,sigma
    x = (data-mu)/sigma
    return x


In [4]:
def fit(train,target):
    global w , l_rate
    for ind in range(len(train)):
        x = train[ind]
        t = target[ind]
        diff = sigmoid(x)-t
        w = w - (l_rate*diff) * x
    
       
def fit_ridge(train,target):
    global w , l_rate,lamda
    reg =w
    lamda = 0.005
    reg = [lamda*wi for wi in reg]
    for ind in range(len(train)):
        x = train[ind]
        t = target[ind]
        diff = sigmoid(x)-t 
        w = w - ((l_rate*diff) * x)+ reg

def fit_lasso(train,target):
    global w , l_rate,lamda
    reg =[]
    lamda = 0.0001
    for wi in w:
        if wi<0:
            reg.append(-1.0*lamda)
        else:
            reg.append(1.0*lamda)
    for ind in range(len(train)):
        x = train[ind]
        t = target[ind]
        diff = sigmoid(x)-t 
        w = w - ((l_rate*diff) * x )+ reg


In [5]:
def predict(test):
    p = []
    for i in range(len(test)):
        prob = sigmoid(test[i])
        if prob>0.5:
            p.append(1)
        else:
            p.append(0)
    return p

def acc(actual,pred):
    y_actu = pd.Series(actual, name='Actual')
    y_pred = pd.Series(pred, name='Predicted')
    cm = pd.crosstab(y_actu, y_pred)
    #print(cm)
    acc = (cm.iloc[0,0]+cm.iloc[1,1])/len(actual)
    p = cm.iloc[0,0]/(cm.iloc[0,0]+cm.iloc[1,0])
    r = cm.iloc[0,0]/(cm.iloc[0,0]+cm.iloc[0,1]) 
    f_score = 2*p*r/(p+r)
    print('acc= ',round(acc,4),'FScore = ',round(f_score,4))

def classify(xtrain,ytrain,xtest,ytest):
    global w,vectors
    w1 = w
    print('wihout regularisation')
    fit(xtrain,ytrain)
    pred = predict(xtest)
    acc(ytest,pred)
    print('weight vector ',w)
    vectors.append(w)
    w = w1
    print('lasso regularisation')
    fit_lasso(xtrain,ytrain)
    pred = predict(xtest)
    acc(ytest,pred)
    print('weight vector ',w)
    vectors.append(w)
    w = w1
    print('ridge regularisation')
    fit_ridge(xtrain,ytrain)
    pred = predict(xtest)
    acc(ytest,pred)
    print('weight vector ',w)
    vectors.append(w)


In [6]:
def logit():
    global w,l_rate
    etas = [0.25,0.05,0.1]
    w1 = w
    for e in etas:
        w = w1
        l_rate = e
        print('learning rate = ',l_rate)
        classify(xtrain,ytrain,xtest,ytest)
        print('\n')


In [7]:
data = pd.read_csv('data_banknote_authentication.csv')
train=data.sample(frac=0.8,random_state=200) #random state is a seed value
test=data.drop(train.index)
xtrain,ytrain = train.iloc[:,:-1],np.asarray(train.iloc[:,-1])
xtest,ytest = test.iloc[:,:-1],np.asarray(test.iloc[:,-1])
mu,sigma = xtrain.mean(),xtrain.std()
xtrain = mean_normalise(xtrain)
xtest = mean_normalise(xtest)

"""adding column of 1s for the constant w0"""
xtrain = np.append(xtrain,[[1]]*xtrain.shape[0],axis=1)
xtest = np.append(xtest,[[1]]*xtest.shape[0],axis=1)


In [8]:
""" init w randomly with values in the range -100 to 100 lrate = 5"""
w = [round(np.random.randint(-10,10)/1.0,4) for _ in range(xtrain.shape[1])]
logit()

""" init w randomly with values in the range -0.6 to 0.6 lrate = 0.05"""
w = [round(np.random.uniform(-0.6,0.6),4) for _ in range(xtrain.shape[1])]
logit()

""" uniform initialistion lr = 0.05"""
w = [1] * xtrain.shape[1]
logit()

""" Gaussian init lr = 5 """
w = [round(np.random.normal(2,0.5),4)for _ in range(xtrain.shape[1])]
logit()


learning rate =  0.25
wihout regularisation
acc=  0.9781 FScore =  0.9804
weight vector  [-5.55005245 -4.74690339 -4.42180775  0.44154523 -1.49      ]
lasso regularisation
acc=  0.9781 FScore =  0.9804
weight vector  [-5.56647781 -4.68011837 -4.38009205  0.44621861 -1.4677    ]
ridge regularisation
acc=  0.7956 FScore =  0.8146
weight vector  [-50.29047036   6.70711178   6.08050844 -18.99864547   2.15      ]


learning rate =  0.05
wihout regularisation
acc=  0.8285 FScore =  0.8449
weight vector  [-8.37135759 -0.93292654  0.50637282 -0.77227656 -0.5115    ]
lasso regularisation
acc=  0.8248 FScore =  0.8421
weight vector  [-8.49274677 -0.85975853  0.61735706 -0.85194082 -0.4542    ]
ridge regularisation
acc=  0.6861 FScore =  0.695
weight vector  [-63.07373401  33.7932965   18.43148856 -48.75797437  14.602     ]


learning rate =  0.1
wihout regularisation
acc=  0.9453 FScore =  0.9521
weight vector  [-6.49263627 -2.64521523 -2.89992231  0.99755039 -1.301     ]
lasso regularisation
ac